# Geoembeddings
---
XXX

In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [31]:
#AoI using assets
aoi = ee.Geometry.Polygon(
[[[5.608534480510099, 46.698303309141494],
[5.608534480510099, 45.80952394852957],
[7.476210261760099, 45.80952394852957],
[7.476210261760099, 46.698303309141494]]])
Map.addLayer(aoi,{},'AOI')
Map.centerObject(aoi, 9)

startDate = '2019-01-01'
endDate = '2020-01-01'

In [32]:
#Sentinel-2
#Cloud masking 
def cloudMask(image):
    scl = image.select('SCL')
    mask = scl.eq(3).Or(scl.gte(7).And(scl.lte(10)))
    return image.updateMask(mask.eq(0))

#Import Sentinel-2 image collection
CollectionS2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
              .filterBounds(aoi) \
              .filterDate(startDate, endDate) \
              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
              .map(cloudMask)

#RGB visualization parameters
visualizationS2 = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 1800,
}

#Visualize RGB median
MedianS2 = CollectionS2.median().clip(aoi)
Map.addLayer(MedianS2, visualizationS2, 'Sentinel-2 | True Color')

In [33]:
dataset = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL') \
    .filterDate(startDate, endDate) \
    .filterBounds(aoi)

clipped = dataset.median().clip(aoi)

visParams = {"min": -0.3, "max": 0.3, "bands": ['A01', 'A32', 'A09']}

Map.addLayer(clipped, visParams, 'embeddings')

In [53]:
dataset = ee.Image('COPERNICUS/CORINE/V20/100m/2018').select("landcover").clip(aoi)
luc = dataset.select("landcover").clip(aoi)
Map.addLayer(luc, {}, "LUC")

In [54]:
# Make the training dataset.
points = luc.sample(
    **{
        "region": aoi,
        "scale": 10,
        "numPixels": 5000,
        "seed": 30,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, "training", False)

In [55]:
# Use these bands for prediction.
bands = clipped.bandNames()

# This property of the table stores the land cover labels.
label = "landcover"

# Overlay the points on the imagery to get training.
training = clipped.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 10}
)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [56]:
# Classify the image with the same bands used for training.
result = clipped.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, "classified")

In [57]:
class_values = luc.get("landcover_class_values").getInfo()
#class_values

In [58]:
class_palette = luc.get("landcover_class_palette").getInfo()
#class_palette

In [59]:
landcover = result.set("classification_class_values", class_values)
landcover = landcover.set("classification_class_palette", class_palette)

In [60]:
Map.addLayer(landcover, {}, "Land cover")
#Map

In [22]:
# Prepare the Image for Clustering
# Select a subset of bands (the dataset has 64 embedding bands)
bands = clipped.bandNames()
# Optionally, use only the first few bands to reduce computation:
# bands = bands.slice(0, 32)
training_image = clipped.select(bands)

# Sample the Image (for clustering input)
# Take random samples within AOI
training_samples = training_image.sample(
    region=aoi,
    scale=10,            # pixel resolution (adjust as needed)
    numPixels=1000,      # number of training samples
    seed=20
)

# Train a K-Means Clusterer
# Number of clusters (adjust as desired)
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training_samples)

# Classify the Image
classified = training_image.cluster(clusterer)

# Display the Results on a Map
Map.addLayer(classified.randomVisualizer(), {}, f'K-Means (k={n_clusters})')